In [ ]:
# Install Ollama and Huggingface packages
%pip install llama-index-llms-ollama llama-index-embeddings-huggingface

In [ ]:
# Use local model
from llama_index.llms.ollama import Ollama
llm = Ollama(model="llama3.2:latest", request_timeout=120)

In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# Load documents
documents = SimpleDirectoryReader('files').load_data()

# Set up local embeddings (you can specify a different model too)
embed_model = HuggingFaceEmbedding(model_name="all-MiniLM-L6-v2")

# Build the index with local embeddings
index = VectorStoreIndex.from_documents(documents, embed_model=embed_model)

# Set up query engine using Ollama
query_engine = index.as_query_engine(llm=llm)

# Query
response = query_engine.query("Summarize each document in a few sentences.")
print(response)


Document summaries:

One text describes a common human-companion relationship, highlighting the unique qualities and roles that dogs play in people's lives. It showcases their versatility as pets and the positive impact they have on owners.

The other text delves into ancient Rome's history and achievements, featuring its architectural marvels, engineering feats, and cultural legacies. It highlights the Roman Empire's vast territories, military prowess, and enduring influence on modern societies.
